In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import shared_utils

from calitp_data_analysis import get_fs
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT, analysis_date

In [2]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [5]:
catalog.speedmap_segments

{'dir': '${gcs_paths.SEGMENT_GCS}', 'stage1': '${speeds_tables.vp_dwell}', 'proxy_stop_times': 'stop_time_expansion/speedmap_stop_times', 'stage2': 'nearest/nearest_vp_speedmap_proxy', 'stage3': 'speedmap/stop_arrivals_proxy', 'stage3b': 'speedmap/stop_arrivals', 'stage4': 'speedmap/speeds', 'trip_stop_cols': ['trip_instance_key', 'stop_sequence', 'stop_sequence1'], 'shape_stop_cols': ['shape_array_key', 'shape_id'], 'stop_pair_cols': ['stop_pair', 'stop_pair_name', 'segment_id'], 'route_dir_cols': ['route_id', 'direction_id'], 'segments_file': 'segment_options/speedmap_segments', 'shape_stop_single_segment': 'rollup_singleday/speeds_shape_speedmap_segments', 'shape_stop_single_segment_detail': 'rollup_singleday/speeds_shape_speedmap_segments_detail', 'excluded_shapes_by_time_of_day': 'rollup_singleday/excluded_shapes_by_time_of_day', 'route_dir_single_segment': 'rollup_singleday/speeds_route_dir_speedmap_segments', 'route_dir_multi_segment': 'rollup_multiday/speeds_route_dir_speedmap_

In [4]:
analysis_date

'2024-12-11'

In [6]:
helpers.import_scheduled_shapes?

Signature:
helpers.import_scheduled_shapes(
    analysis_date: str,
    filters: tuple = None,
    columns: list = ['shape_array_key', 'geometry'],
    get_pandas: bool = True,
    crs: str = 'EPSG:3310',
) -> Union[geopandas.geodataframe.GeoDataFrame, dask_geopandas.core.GeoDataFrame]
Docstring: Import shapes.
File:      ~/data-analyses/rt_segment_speeds/segment_speed_utils/helpers.py
Type:      function

In [7]:
shapes = helpers.import_scheduled_shapes(analysis_date)

In [9]:
helpers.import_scheduled_trips?

Signature:
helpers.import_scheduled_trips(
    analysis_date: str,
    filters: tuple = None,
    columns: list = ['gtfs_dataset_key', 'name', 'trip_id', 'shape_id', 'shape_array_key', 'route_id', 'route_key', 'direction_id', 'route_short_name'],
    get_pandas: bool = True,
) -> Union[pandas.core.frame.DataFrame, dask.dataframe.core.DataFrame]
Docstring:
Get scheduled trips info (all operators) for single day, 
and keep subset of columns.
File:      ~/data-analyses/rt_segment_speeds/segment_speed_utils/helpers.py
Type:      function

In [13]:
trips = helpers.import_scheduled_trips(analysis_date, columns=['shape_array_key', 'shape_id', 'route_id',
                                                               'route_short_name', 'gtfs_dataset_key']).drop_duplicates()

In [14]:
shapes.merge(trips, on='shape_array_key')

,shape_array_key,geometry,shape_id,route_id,route_short_name,schedule_gtfs_dataset_key
0,13cb51c5ed7990a71a227d7d096426b9,"LINESTRING (294120.912 -448955.567, 294061.160...",p_1432525,355,1,ebc783bace70899492d6206c352547d6
1,229051b21f5e16ec5841ecabf98a12c7,"LINESTRING (-78208.700 215570.571, -77955.213 ...",p_1426803,126,None,96a5b8c5bc22b3936ddee01cac7b5fb6
2,f08b835c09fe1d9e38199ad09c9327d8,"LINESTRING (91775.849 -41429.921, 91784.599 -4...",p_1435920,76755,None,cb8a465cffec67c8fd90f31b389ed4c3
3,6be599a186ee46f51fe77bb0bc0ffc50,"LINESTRING (-118608.801 41853.097, -118632.536...",8377,3431,13,63029a23cb0e73f2a5d98a345c5e2e40
4,3c2b4b3de0eef26767c626504e7a839e,"LINESTRING (-210809.438 -23394.433, -210787.18...",1540061,154,154,aea4108997c66a74fbdae27b34b69fde
...,...,...,...,...,...,...
7564,f90c7ca85f79cec1cf718ffe3a5ea804,"LINESTRING (173733.400 -444977.431, 173388.307...",8577_shp,50,None,84d3c3507a4c8df851d935b63908bbd5
7565,4e41a760292c44d8c07b5e187903d3cb,"LINESTRING (169600.675 -449977.512, 169620.127...",8582_shp,30,None,84d3c3507a4c8df851d935b63908bbd5
7566,8bf6f14bb04f53e24929661cf9aee280,"LINESTRING (176938.852 -444188.785, 176853.905...",8659_shp,10,None,84d3c3507a4c8df851d935b63908bbd5
7567,7ed83def41f1f965236ca51c4e059d5b,"LINESTRING (168680.735 -437987.848, 168658.352...",8656_shp,10,None,84d3c3507a4c8df851d935b63908bbd5


In [24]:
path = f"{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet"

In [25]:
path

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/rollup_singleday/speeds_shape_speedmap_segments_detail_2024-12-11.parquet'

In [26]:
gdf = gpd.read_parquet(path)

In [27]:
gdf.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'route_id',
       'direction_id', 'stop_pair', 'stop_pair_name', 'segment_id',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph', 'name',
       'caltrans_district', 'organization_source_record_id',
       'organization_name', 'base64_url', 'geometry', 'n_trips_sch',
       'trips_hr_sch', 'route_short_name'],
      dtype='object')

could presumably iterate over `schedule_gtfs_dataset_key` and append to one gdf to have one file...